In [60]:
import numpy as np
import timeit

from scipy.sparse import csr_matrix
from graspologic.inference import latent_position_test
from graspologic.embed import AdjacencySpectralEmbed, LaplacianSpectralEmbed
from graspologic.simulations import er_np, sbm, rdpg
from graspologic.utils import symmetrize
from graspologic.plot import heatmap, pairplot
from graspologic.align import OrthogonalProcrustes

from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

from sknetwork.data import erdos_renyi, painters, movie_actor
from sknetwork.embedding import Spectral, SVD
from sknetwork.visualization import svg_graph, svg_digraph, svg_bigraph
from IPython.display import SVG

In [71]:
#ValueError when d<n 
X = np.random.uniform(low=0.1, high=1, size=(5, 4))
norm = np.linalg.norm(X)
X = X/norm
A = csr_matrix(X)
print(A)
ase = AdjacencySpectralEmbed(n_components=4, check_lcc = False)
X1 = ase.fit_transform(A)

  (0, 0)	0.13487956335236997
  (0, 1)	0.2984579184080641
  (0, 2)	0.3458138356983496
  (0, 3)	0.22344399078312507
  (1, 0)	0.08596189367855021
  (1, 1)	0.3061188027468812
  (1, 2)	0.07787355871450072
  (1, 3)	0.16786696030302783
  (2, 0)	0.06076780504735486
  (2, 1)	0.16290551409402187
  (2, 2)	0.29102449020761467
  (2, 3)	0.315542789449708
  (3, 0)	0.2585473500656868
  (3, 1)	0.2597367672557316
  (3, 2)	0.09353178147051838
  (3, 3)	0.20548612032750704
  (4, 0)	0.23188700916548285
  (4, 1)	0.07716811483873472
  (4, 2)	0.35008431229633846
  (4, 3)	0.07624004715791148


ValueError: Found array with 4 feature(s) (shape=(5, 4)) while a minimum of 5 is required.

In [72]:
#ValueError when d>n 
X = np.random.uniform(low=0.1, high=1, size=(5, 6))
norm = np.linalg.norm(X)
X = X/norm
A = csr_matrix(X)
print(A)
ase = AdjacencySpectralEmbed(n_components=6, check_lcc = False)
X1 = ase.fit_transform(A)

  (0, 0)	0.1306751441138062
  (0, 1)	0.20364540826480007
  (0, 2)	0.22411846642689642
  (0, 3)	0.11012971184413588
  (0, 4)	0.108653158914982
  (0, 5)	0.2660147866736775
  (1, 0)	0.17367023608269738
  (1, 1)	0.26009658546469777
  (1, 2)	0.23280626776442184
  (1, 3)	0.04702279759955788
  (1, 4)	0.17982923005635604
  (1, 5)	0.10772892607449097
  (2, 0)	0.074877834979874
  (2, 1)	0.04746882305971947
  (2, 2)	0.23214826080680415
  (2, 3)	0.11493298632814013
  (2, 4)	0.16293908259278617
  (2, 5)	0.1420771607877195
  (3, 0)	0.09555524644178184
  (3, 1)	0.038519405165890704
  (3, 2)	0.28616026291808705
  (3, 3)	0.2974830741066051
  (3, 4)	0.05464570556124078
  (3, 5)	0.25818197252837394
  (4, 0)	0.10504213101976805
  (4, 1)	0.16649536999052547
  (4, 2)	0.20883356973879594
  (4, 3)	0.27632500330231197
  (4, 4)	0.22306727246154187
  (4, 5)	0.13243672572484402


ValueError: Found array with 5 sample(s) (shape=(5, 6)) while a minimum of 6 is required.

# Thus when using csr_matrix n must equal d

In [73]:
#error when n_components of ASE >= n
X = np.random.uniform(low=0.1, high=1, size=(5, 5))
norm = np.linalg.norm(X)
X = X/norm
A = csr_matrix(X)
print(A)
ase = AdjacencySpectralEmbed(n_components=5, check_lcc = False)
X1 = ase.fit_transform(A)

  (0, 0)	0.24904097230349798
  (0, 1)	0.19616304283182573
  (0, 2)	0.20166160524143806
  (0, 3)	0.06379374965634202
  (0, 4)	0.04488738389238659
  (1, 0)	0.2533314958969232
  (1, 1)	0.20941812848437863
  (1, 2)	0.3153334687814674
  (1, 3)	0.28797273592239364
  (1, 4)	0.04779200401153193
  (2, 0)	0.17772413247845467
  (2, 1)	0.23373077301229073
  (2, 2)	0.07004087533485218
  (2, 3)	0.04407366982750648
  (2, 4)	0.19560095250610035
  (3, 0)	0.16759087519597432
  (3, 1)	0.07077784446987202
  (3, 2)	0.14991259577525043
  (3, 3)	0.2836469822016937
  (3, 4)	0.2066009511851916
  (4, 0)	0.09165361500358332
  (4, 1)	0.03646387628730389
  (4, 2)	0.2058998082247048
  (4, 3)	0.30458237990355186
  (4, 4)	0.32424954258251


ValueError: n_components must be strictly < min(X.shape).

In [74]:
X = np.random.uniform(low=0.1, high=1, size=(5, 5))
norm = np.linalg.norm(X)
X = X/norm
A = csr_matrix(X)
print(A)
ase = AdjacencySpectralEmbed(n_components=4, check_lcc = False)
X1 = ase.fit_transform(A)
norm1 = np.linalg.norm(X1, axis=0)
X1 = X1/norm1

  (0, 0)	0.3125181379488869
  (0, 1)	0.1346356803081745
  (0, 2)	0.08235935432509696
  (0, 3)	0.26583651396597435
  (0, 4)	0.17124225508129143
  (1, 0)	0.12287815833439497
  (1, 1)	0.07503320719826263
  (1, 2)	0.3469345209182601
  (1, 3)	0.10158129397049089
  (1, 4)	0.20401595127736802
  (2, 0)	0.08022113044644086
  (2, 1)	0.3180919585552152
  (2, 2)	0.18634302030246688
  (2, 3)	0.10654800550804673
  (2, 4)	0.10436516091612721
  (3, 0)	0.11252009613022583
  (3, 1)	0.225414526548256
  (3, 2)	0.17055483909688732
  (3, 3)	0.1617419067713133
  (3, 4)	0.3242106680890201
  (4, 0)	0.26454570153582535
  (4, 1)	0.07642186323880346
  (4, 2)	0.0782078589332615
  (4, 3)	0.08252556971937101
  (4, 4)	0.3274499274659511


In [75]:
#error with aligning when n_components of ASE<n
aligner_OP1 = OrthogonalProcrustes()
X_prime_OP1 = aligner_OP1.fit_transform(X, X1)
Q1 = aligner_OP1.Q_
norm_ASE = np.linalg.norm(X@Q1-X1, ord='fro')

ValueError: Found array with dim 3. Estimator expected <= 2.